In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/malayalam-news/HackFake Database for the Hackathon - Sheet1.csv


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
tf.get_logger().setLevel('ERROR')

In [3]:
#Read dataset
FILEPATH='/kaggle/input/malayalam-news'
df_train=pd.read_csv(os.path.join(FILEPATH,'HackFake Database for the Hackathon - Sheet1.csv'))[:500]
df_train = df_train.drop(['Website (source)', 'Timeline/Date','Title'], axis=1)[2:].reset_index(drop=True)
df_train=df_train.dropna(axis=0)

In [4]:
df_train.columns

Index(['Heading of News Article', 'Text of News Article', 'Hate speech',
       'Misleading', 'Disinformation', 'Rumor/Hoax', 'Sensationalism',
       'Credible'],
      dtype='object')

In [5]:
#'Heading of News Article'

In [6]:
X=df_train[['Text of News Article']]
#'Heading of News Article'
y=df_train[['Hate speech','Misleading','Disinformation','Rumor/Hoax','Sensationalism']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
y_train=np.array(y_train).astype('int')
y_test=np.array(y_test).astype('int')

In [18]:
#Testing using two features
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import regularizers,layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout


# Input sentence
input_sentence1 = X_train.iloc[2]

# Preprocessing layer
preprocessor_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2"
)

# Encoder layer
encoder_layer = hub.KerasLayer(
    "https://tfhub.dev/google/LaBSE/2",
    trainable=False
)

# LSTM layer
lstm_layer = tf.keras.layers.LSTM(units=64,return_sequences=False)

# Preprocess the input sentence
preprocessed_text1 = preprocessor_layer(input_sentence1)

# Get the embeddings for each sentence
embedded_text1 = encoder_layer(preprocessed_text1)['sequence_output']
print(embedded_text1.shape)
# Pass the embeddings through LSTM layer
lstm_output1 = lstm_layer(embedded_text1)
#lstm_output2=layers.GlobalAveragePooling1D()(lstm_output1)
output_layer1 = layers.Dense(units=5, activation='sigmoid', name='output')(lstm_output1)[0]

(1, 128, 768)


In [19]:
lstm_output1

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[ 0.36466417, -0.06235673, -0.49386132,  0.3998707 ,  0.28263465,
         0.21895479,  0.8492675 ,  0.17877989,  0.2605363 ,  0.16537982,
        -0.1268451 , -0.8086826 ,  0.44012436,  0.08345265,  0.16388373,
        -0.43604088,  0.274813  ,  0.09985818, -0.27723584, -0.53470767,
        -0.35864088,  0.7187821 , -0.46535867,  0.34102157,  0.641211  ,
         0.72934496, -0.25164443,  0.2275606 ,  0.07320295, -0.81434464,
         0.02539192, -0.16988501,  0.14795533,  0.2443287 , -0.10293993,
         0.34878716,  0.74245745, -0.4020983 ,  0.5210856 , -0.6425313 ,
         0.29388773, -0.7414161 ,  0.2993094 ,  0.41533205,  0.3716709 ,
         0.20997384,  0.86726815,  0.19462824,  0.13151288,  0.5888414 ,
         0.5510913 ,  0.04660117, -0.31519857, -0.28027093, -0.30018297,
         0.3896256 ,  0.45770136,  0.53884375, -0.08484893, -0.0480017 ,
         0.38754663, -0.7438719 , -0.16370496,  0.08571001]],
      dtype=f

In [8]:
lstm_output2

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[-0.50177264,  0.14210251,  0.02767043, -0.08150056, -0.29197174,
         0.01224235, -0.26337665,  0.08613162,  0.67611754,  0.24982798,
         0.3260829 , -0.17401351, -0.72235537,  0.5995993 , -0.02389341,
         0.04326465,  0.04119503,  0.49490112, -0.7706332 ,  0.25575194,
         0.15110743, -0.64067924, -0.00380334, -0.6688314 , -0.5703141 ,
        -0.47973478, -0.09044906,  0.10038604, -0.03906211, -0.28935   ,
        -0.19304404,  0.3013301 , -0.12267329, -0.4191303 , -0.20410295,
         0.66920054,  0.18628544, -0.22777057, -0.5157732 , -0.28338918,
        -0.34805262,  0.7066706 ,  0.31185448, -0.43620026, -0.5106785 ,
        -0.41335738, -0.01533353,  0.05476393, -0.2721781 , -0.27166736,
        -0.07997006,  0.233642  , -0.0954653 ,  0.17356649, -0.4820694 ,
         0.27877942,  0.63666946,  0.01430285,  0.2109234 , -0.02518149,
        -0.07357725, -0.11714554, -0.25975347,  0.40260035]],
      dtype=f

In [9]:
lstm_output1

<tf.Tensor: shape=(1, 128, 64), dtype=float32, numpy=
array([[[-0.10932609,  0.27344152, -0.07559771, ..., -0.22532369,
         -0.10157155,  0.16877033],
        [-0.33951098,  0.2583799 , -0.01795679, ..., -0.34004822,
         -0.14837994,  0.12447042],
        [-0.46045884,  0.2699605 , -0.05787717, ..., -0.39572194,
         -0.18516226,  0.04873728],
        ...,
        [-0.55945504,  0.08836748, -0.21558997, ...,  0.16951804,
         -0.3344891 ,  0.57873327],
        [-0.48331943,  0.0888402 , -0.18297471, ...,  0.24282566,
         -0.30490017,  0.6174341 ],
        [-0.31528634,  0.29826215, -0.2245811 , ...,  0.03557097,
         -0.2959899 ,  0.733321  ]]], dtype=float32)>

In [10]:
encoder_layer(preprocessed_text1)['sequence_output']

<tf.Tensor: shape=(1, 128, 768), dtype=float32, numpy=
array([[[-0.5687288 , -0.78794324, -0.07094856, ..., -0.977919  ,
         -0.9961291 , -0.81047994],
        [ 0.14578322, -1.2176663 , -0.40914422, ..., -0.9570222 ,
         -1.068472  , -0.97340447],
        [ 0.20238103, -1.4000866 , -0.2963425 , ..., -0.99640495,
         -1.1418309 , -0.6948013 ],
        ...,
        [ 0.30593705, -1.7080377 , -0.5031715 , ..., -0.6436213 ,
         -0.968608  , -1.289979  ],
        [ 0.07543968, -1.4575853 , -0.08249031, ..., -0.87103367,
         -0.91912663, -1.0585786 ],
        [-0.5687289 , -0.7879431 , -0.07094832, ..., -0.97791934,
         -0.9961291 , -0.81047994]]], dtype=float32)>

In [11]:
X_train.iloc[2]

Text of News Article    ഒരു കാലത്ത് നിറഞ്ഞു നിന്നിരുന്ന നാടകങ്ങളിലൂടെ ...
Name: 157, dtype: object

In [12]:
lstm_output1

<tf.Tensor: shape=(1, 128, 64), dtype=float32, numpy=
array([[[-0.10932609,  0.27344152, -0.07559771, ..., -0.22532369,
         -0.10157155,  0.16877033],
        [-0.33951098,  0.2583799 , -0.01795679, ..., -0.34004822,
         -0.14837994,  0.12447042],
        [-0.46045884,  0.2699605 , -0.05787717, ..., -0.39572194,
         -0.18516226,  0.04873728],
        ...,
        [-0.55945504,  0.08836748, -0.21558997, ...,  0.16951804,
         -0.3344891 ,  0.57873327],
        [-0.48331943,  0.0888402 , -0.18297471, ...,  0.24282566,
         -0.30490017,  0.6174341 ],
        [-0.31528634,  0.29826215, -0.2245811 , ...,  0.03557097,
         -0.2959899 ,  0.733321  ]]], dtype=float32)>

In [ ]:
lstm_output1

In [ ]:
embedded_text1

In [ ]:
preprocessed_text1['input_word_ids']

In [26]:
#Testing using single features
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import regularizers,layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout


# Input sentence
input_sentence = ['ലാലേട്ടൻ ന്റെ അടിപൊളി രാജാവിന്റെ മകൻ']

# Preprocessing layer
preprocessor_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2"
)

# Encoder layer
encoder_layer = hub.KerasLayer(
    "https://tfhub.dev/google/LaBSE/2",
    trainable=False
)

# LSTM layer
lstm_layer = tf.keras.layers.LSTM(units=64, return_sequences=True)

# Preprocess the input sentence
preprocessed_text = preprocessor_layer(input_sentence)

# Get the embeddings for each sentence
embedded_text = encoder_layer(preprocessed_text)["default"]
print(embedded_text.shape)
embedded_text = tf.expand_dims(embedded_text, axis=1)
print(embedded_text.shape)
# Pass the embeddings through LSTM layer
lstm_output = lstm_layer(embedded_text)

output_layer = layers.Dense(units=5, activation='sigmoid', name='output')(lstm_output)[0]

(1, 768)
(1, 1, 768)


In [ ]:
preprocessed_text['input_word_ids'][0]

In [ ]:
preprocessed_text1['input_word_ids'][0]

In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers,layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization,Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, BatchNormalization, Dropout

#Custom CallBack and Scheduler
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=15,
    min_delta=0.001,
    restore_best_weights=True,
)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=X.shape[0]*20,
  decay_rate=1,
  staircase=False)


def tf_model():
    # Input layer
    input_layer = Input(shape=(), dtype=tf.string, name='text_input')

    # Preprocessor layer
    preprocessor_layer = hub.KerasLayer(
        "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2"
    )
    preprocessed_text = preprocessor_layer(input_layer)

    # Encoder layer
    encoder_layer = hub.KerasLayer(
        "https://tfhub.dev/google/LaBSE/2",
        trainable=False
    )

    embedded_text = encoder_layer(preprocessed_text)["sequence_output"]
    # Pass the embeddings through LSTM layer
    lstm_layer = tf.keras.layers.LSTM(units=64,return_sequences=False)
    lstm_output = lstm_layer(embedded_text)
    # For example, add a Dense layer for classification
    output_layer = layers.Dense(units=5, activation='sigmoid', name='output')(lstm_output)

    
    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    return model

# Create the model
model = tf_model()

# Display the model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text_input (InputLayer)     [(None,)]                    0         []                            
                                                                                                  
 keras_layer_14 (KerasLayer  {'input_mask': (None, 128)   0         ['text_input[0][0]']          
 )                           , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                            

In [27]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='binary_crossentropy',
        metrics=['acc']
)

In [30]:
model.fit(X_train,y_train,epochs=10,callbacks=[early_stopping],validation_data=[X_test,y_test],batch_size=50)

Epoch 1/10
7/7 [==============================] - 124s 19s/step - loss: 0.5036 - acc: 0.3223 - val_loss: 0.6133 - val_acc: 0.2909
Epoch 2/10
7/7 [==============================] - 119s 18s/step - loss: 0.4652 - acc: 0.4036 - val_loss: 0.6258 - val_acc: 0.2667
Epoch 3/10
7/7 [==============================] - 119s 18s/step - loss: 0.4234 - acc: 0.3765 - val_loss: 0.6322 - val_acc: 0.2606
Epoch 4/10
7/7 [==============================] - 162s 25s/step - loss: 0.3692 - acc: 0.3825 - val_loss: 0.6540 - val_acc: 0.2727
Epoch 5/10
7/7 [==============================] - 121s 18s/step - loss: 0.3355 - acc: 0.4337 - val_loss: 0.7133 - val_acc: 0.2970
Epoch 6/10
7/7 [==============================] - 121s 18s/step - loss: 0.2904 - acc: 0.4458 - val_loss: 0.7772 - val_acc: 0.3394
Epoch 7/10
7/7 [==============================] - 122s 18s/step - loss: 0.2785 - acc: 0.4006 - val_loss: 0.7668 - val_acc: 0.2970
Epoch 8/10
7/7 [==============================] - 121s 18s/step - loss: 0.2671 - acc: 0.50

In [32]:
model.predict(X_train.iloc[5])

1/1 [==============================] - 2s 2s/step


array([[0.7995297 , 0.09299427, 0.04338548, 0.05100969, 0.11281256]],
      dtype=float32)

In [34]:
y_train[5]

array([1, 0, 0, 0, 0])

In [41]:
model.predict(X_test.iloc[10])

1/1 [==============================] - 0s 327ms/step


array([[0.44847015, 0.69600606, 0.7582715 , 0.283104  , 0.9420537 ]],
      dtype=float32)

In [42]:
y_test[10]

array([0, 0, 0, 0, 1])

In [43]:
model.predict(['പലെസ്ടിനെ തീവ്രവാദികളുടെ രാജ്യമാണ്'])

1/1 [==============================] - 0s 366ms/step


array([[0.5278916 , 0.3669245 , 0.3528354 , 0.6818203 , 0.59544784]],
      dtype=float32)

In [44]:
model.predict(['എന്നും എന്നെന്നും പ്രേക്ഷകർക്ക് പ്രിയങ്കരർ ആണ് മഞ്ജുവും ദിലീപും. ആദ്യ സിനിമയിലെ പ്രണയജോഡികൾ ജീവിതത്തിലും ഒന്നായപ്പോൾ ഇരുവരോടുമുള്ള ഇഷ്ടവും ആരാധനയും കൂടുകയായിരുന്നു പ്രേക്ഷകർക്ക്. സംതൃപ്തകരമായ ജീവിതം നയിക്കുന്നതിന്റെ ഇടയിൽ ഇരുവർക്കും പക്ഷെ ജീവിതത്തിൽ വേർപിരിയേണ്ടി വന്നു. ഇന്നും അജ്ഞാതമായ കാരണമായി അവശേഷിക്കുക്കയാണ് ഇവരുടെ ഡിവോഴ്സ് സ്റ്റോറി'])

1/1 [==============================] - 0s 352ms/step


array([[0.32871833, 0.03807576, 0.03816624, 0.02055259, 0.71818596]],
      dtype=float32)

In [48]:
model.save('lstm.keras')

In [ ]:
def func(Text):
    Contents=['Hate','Misleading','Disinformation','Rumor','Sensationalism']
    Scores=model_final.predict(Text)
    Card={}
    for score,content in zip(Scores[0],Contents):
        Card[content]=str(score)
    return Card